In [ ]:
print("ok")

In [9]:
import pytesseract
from PIL import Image
from transformers import pipeline
from langid import classify
import regex

def analyze_code_snippet(image_path):
    # Step 1: Extract text from the image using OCR
    try:
        image = Image.open(image_path)
        extracted_text = pytesseract.image_to_string(image)
        print(f"Extracted Text:\n{extracted_text}")
    except Exception as e:
        print("Error extracting text:", e)
        return

    # Step 2: Identify programming language
    try:
        code_detector = pipeline("text-classification", model="huggingface/CodeBERTa-language-id")
        language_prediction = code_detector(extracted_text)
        language = language_prediction[0]['label']
        confidence = language_prediction[0]['score']
        code_explainer = pipeline("summarization", model="ashwinR/CodeExplainer")
        explanation = code_explainer(extracted_text)[0]['summary_text']
        print(f"Code Explanation:\n{explanation}")
        print(f"Detected Language: {language} (Confidence: {confidence:.2%})")
    except Exception as e:
        print("Error identifying language:", e)
        language = None
        confidence = None

    # Step 3: Search for author's name
    author_pattern = r'author\s*[:=]\s*["\']?([\w\s]+)["\']?'
    author_match = regex.search(author_pattern, extracted_text, regex.IGNORECASE)
    author = author_match.group(1) if author_match else "Unknown"
    print(f"Author: {author}")

    

    # Step 4: Show accuracy metrics
    if confidence is not None:
        accuracy_score = confidence * 100
        print(f"Accuracy: {accuracy_score:.2f}%")
    else:
        print("Accuracy: N/A")

    return {"language": language, "author": author, "confidence": confidence}

# Example usage
if __name__ == "__main__":
    image_path = "D:\AI-Projects\code-classification\3DTopia_LGM_main_part1.png"  # Replace with your image path
    analyze_code_snippet(image_path)


Extracted Text:
Fie kat Sewch View Project fan Toate
PUGS KHHOOPAT SSP) oioiao cu wi8elo-o-leelie aria =|
l# File: factorial.py
# Simple example program to calculate the factorial of a number
# that is inputted by the user

# returns the factorial of the argument "number"
def factorial(number) :
product = 1 ;

for i in range(number) :
product = product * (i+1)

number = input("Enter a non-negative integer to take the factorial of: ")

ami, ens:
ter [re | eink | tet le

a 5




Some weights of the model checkpoint at huggingface/CodeBERTa-language-id were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu
Your max_length is set to 200, but your input_length is only 165. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=82)


Code Explanation:
 (
1. Create a function factorial that will take the factorial of a number that is inputted by the user.
2. In the code we will use the input() function of the program.
3. At first we will ask the user to enter a non-negative integer to take the product of the number.
4. In while loop will loop through the range of numbers and multiply them by one.
5. For each number in the range, we will calculate the product by 1.
6. At last we will take a number from the user's inputted user's choice.

Detected Language: python (Confidence: 59.05%)
Author: Unknown
Accuracy: 59.05%


In [1]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
%pip install pytesseract

In [ ]:
%pip install transformers torch 

In [ ]:
%pip install langid

In [6]:
%pip install regex

Note: you may need to restart the kernel to use updated packages.
